In [13]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import h5py
# Define parameters
L = 1.0       # Length of the rod
Nx = 50       # Number of spatial points
dx = L / Nx   # Space step
T = 0.2       # Total time
Nt = 500     # Number of time steps
dt = T / Nt   # Time step
# Initial condition: Gaussian pulse
x = np.linspace(0, L, Nx)
u0 = np.exp(-100 * (x - L / 2) ** 2)  # Gaussian initial profile
u0[0] = 0
u0[-1] = 0

In [12]:
def heat_eq(t, u):
    u = u.reshape((Nx,))
    du_dt = np.zeros_like(u)
    for i in range(1, Nx - 1):
        du_dt[i] = alpha * (u[i - 1] - 2 * u[i] + u[i + 1]) / dx**2
    return du_dt

In [24]:
for i in range(1000):
    if i % 100 == 0:
        print(i)
    alpha = np.random.uniform(0.1, 0.5) 
    # Stability condition (for explicit scheme)
    s = alpha * dt / dx**2
    if s > 0.5:
        print("Warning: Stability condition violated! Reduce dt or increase dx.")
        continue
    solution = solve_ivp(heat_eq, [0, T], u0, t_eval=np.linspace(0, T, Nt), method="RK45")
    with h5py.File('data.h5', 'a') as f:
        f.create_group(f"{i}")
        f[f"{i}"].create_dataset('u', data=solution.y)
        f[f"{i}"].create_dataset('alpha', data=alpha)
        f.close()
        

0
100
200
300
400
500
600
700
800
900


In [25]:
from scipy.stats import skewnorm

with h5py.File('data.h5', 'a') as f:
    for i in f.keys():
        u = f[i]['u'][:]
        noise = skewnorm.rvs(a=1, scale=0.4, size=u.shape)
        f[i].create_dataset('u_noisy', data=u + noise)
    f.close()

In [26]:
with h5py.File("data.h5", "r") as f:
    print(f["0"].keys())

<KeysViewHDF5 ['alpha', 'u', 'u_noisy']>


In [27]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import animation
from tqdm import tqdm


def visualize_burgers(xcrd, data, path):
    """
    This function animates the Burgers equation

    Args:
    path : path to the desired file
    param: PDE parameter of the data shard to be visualized
    """
    fig, ax = plt.subplots()
    ims = []

    for i in tqdm(range(data.shape[0])):
        if i == 0:
            im = ax.plot(xcrd, data[i].squeeze(), animated=True, color="blue")
        else:
            im = ax.plot(xcrd, data[i].squeeze(), animated=True, color="blue")
        ims.append([im[0]])

    ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True, repeat_delay=1000)

    writer = animation.PillowWriter(fps=15, bitrate=1800)
    ani.save(path, writer=writer)
    plt.close(fig)

In [29]:
import random
with h5py.File("data.h5", "r") as f:
    l = random.choices(list(f.keys()), k=3)
    for i in l:
        u = f[i]['u_noisy'][:]
        visualize_burgers(x, u.T, f"gifs/{i}_noisy.gif")
        u = f[i]['u'][:]
        visualize_burgers(x, u.T, f"gifs/{i}.gif")
    f.close()

100%|██████████| 500/500 [00:00<00:00, 3244.60it/s]
